In [ ]:
### Training logistic Regression on the training set with SEVERITY_CLASS as the target variable
### steps followed:
# training the model
# using CV-HPT 
# trying cv-hpt best model on holdout set

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from utils import utils

In [2]:
df=pd.read_csv('final_ohc_wto_sde_train.csv')

In [3]:
df

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20080506X00599,1,0.012294,0.005339,0.086957,0.00,0.000000,0.25,0.324324,0.225806,...,0,0,0,1,0,0,0,1,1,0
1,20090701X43709,1,0.016400,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.129032,...,0,1,0,0,0,1,0,0,1,0
2,20080827X01344,2,0.014951,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,0,1
3,20090811X65603,1,0.021714,0.005339,0.142068,0.03,0.000000,0.25,0.783784,0.258065,...,0,0,1,0,0,1,0,0,1,0
4,20090608X30506,1,0.013502,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.193548,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12426,20120710X40020,1,0.014951,0.005339,0.095238,0.00,0.233333,0.25,0.783784,0.161290,...,0,0,0,1,0,0,0,1,1,0
12427,20210414102909,1,0.012294,0.005339,0.099576,0.00,0.000000,0.25,0.810811,0.193548,...,0,0,0,1,0,0,0,1,1,0
12428,20120823X95947,1,0.007512,0.002482,0.135266,0.00,0.000000,0.25,0.783784,0.000000,...,0,0,0,1,0,0,0,1,1,0
12429,20080902X01358,1,0.025096,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,1,0


In [4]:
target_column=['severity_class']
x_train, x_test, y_train, y_test=utils.dataset_traintest_split(df, target_column)

In [5]:
x_train = x_train.drop(columns=['ev_id','damage','ev_highest_injury'])
x_test = x_test.drop(columns=['ev_id','damage','ev_highest_injury'])

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, 
                            random_state=42,
                            class_weight='balanced')
rf.fit(x_train, y_train)

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [7]:
y_pred = rf.predict(x_test)

In [8]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, balanced_accuracy_score

In [9]:
accuracy = accuracy_score(y_test, y_pred)
balance = balanced_accuracy_score(y_test, y_pred) 
precision = precision_score(y_test, y_pred, average='weighted')  
recall = recall_score(y_test, y_pred, average='weighted')        
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Balanced Accuracy: {balance:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.67
Balanced Accuracy: 0.14
Precision: 0.52
Recall: 0.67
F1 Score: 0.57


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
from sklearn.model_selection import KFold, GridSearchCV

In [12]:
x = df.drop(columns=['severity_class','ev_id','damage','ev_highest_injury'])
y = df[['severity_class']]
k = 5
fold_size = len(x) // k
for i in range(k):
    x_test2 = x.iloc[i * fold_size : (i + 1) * fold_size]
    y_test2= y.iloc[i * fold_size : (i + 1) * fold_size]
    x_train2 = pd.concat([x.iloc[:i * fold_size], x.iloc[(i + 1) * fold_size:]])
    y_train2 = pd.concat([y.iloc[:i * fold_size], y.iloc[(i + 1) * fold_size:]])

In [13]:
cv = KFold(n_splits=5, shuffle=False)  
rf = RandomForestClassifier(random_state=42)

params = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [None, 10, 20],      
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],    
    'class_weight': ['balanced']      
}

grid_search1 = GridSearchCV(
    estimator=rf,        
    param_grid=params,   
    scoring='balanced_accuracy',
    cv=cv,
    verbose=2,           
    n_jobs=4,           
)

grid_search1.fit(x_train2, y_train2)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(random_state=42), n_jobs=4,
             param_grid={'class_weight': ['balanced'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='balanced_accuracy', verbose=2)

In [15]:
best_params = grid_search1.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


In [19]:
rf = RandomForestClassifier(
    class_weight='balanced',
    max_depth=10,
    min_samples_leaf=4, 
    min_samples_split=10,
    n_estimators=100
)

rf.fit(x_train2, y_train2)
y1_pred1 = rf.predict(x_test2)

balance = balanced_accuracy_score(y_test2, y1_pred1)
print(balance*100)

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


27.444617882060868


In [20]:
a, b, c, d, e = utils.print_metrics(y_test2, y1_pred1)

Accuracy: 0.50
Balanced Accuracy: 0.27
Precision: 0.62
Recall: 0.50
F1 Score: 0.53


In [21]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
dh

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20200819X80407,1,0.007710,0.005339,0.039732,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,1,0,0,1,0
1,20170704X12341,1,0.018236,0.005339,0.411220,0.400000,0.000000,0.25,0.783784,0.322581,...,0,1,0,0,0,0,0,1,1,0
2,20141107X32003,1,0.014458,0.005339,0.089816,0.000000,0.000000,0.25,0.783784,0.290323,...,0,1,0,0,0,0,0,1,1,0
3,20090202X02717,1,0.012294,0.005339,0.093661,0.000000,0.333333,0.25,0.810811,0.290323,...,0,0,0,1,0,1,0,0,1,0
4,20100806X95014,1,0.015434,0.000161,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,20220822105779,1,0.007705,0.005339,0.086957,0.040000,0.056667,0.25,0.783784,0.161290,...,0,1,0,0,0,1,0,0,1,0
3104,20130711X70752,1,0.012294,0.005339,0.027605,0.083333,0.000000,0.25,0.783784,0.387097,...,0,0,1,0,0,1,0,0,1,0
3105,20201026102189,1,0.060355,0.001768,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,1,0,0,0,0,1,0,0,1,0
3106,20160621X92619,1,0.011811,0.004447,0.517993,0.000000,0.000000,0.25,0.783784,0.096774,...,0,0,0,1,0,0,0,1,1,0


In [22]:
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["severity_class"]

In [23]:
y_h_pred = rf.predict(x_h)

In [24]:
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.51
Balanced Accuracy: 0.26
Precision: 0.61
Recall: 0.51
F1 Score: 0.53


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
### Training logistic Regression on the training set with DAMAGE as the target variable
### steps followed:
# training the model
# using CV-HPT 
# trying cv-hpt best model on holdout set

In [25]:
target_column=['damage']
x_train, x_test, y_train, y_test=utils.dataset_traintest_split(df, target_column)

In [26]:
x_train = x_train.drop(columns=['ev_id','severity_class','ev_highest_injury'])
x_test = x_test.drop(columns=['ev_id','severity_class','ev_highest_injury'])

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, 
                            random_state=42,
                            class_weight='balanced')
rf.fit(x_train, y_train)

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [28]:
y_pred2 = rf.predict(x_test)

a, b, c, d, e = utils.print_metrics(y_test,y_pred2)

Accuracy: 0.89
Balanced Accuracy: 0.40
Precision: 0.85
Recall: 0.89
F1 Score: 0.86


In [29]:
x = df.drop(columns=['severity_class','ev_id','damage','ev_highest_injury'])
y = df[['damage']]
k = 5
fold_size = len(x) // k
for i in range(k):
    x_test2 = x.iloc[i * fold_size : (i + 1) * fold_size]
    y_test2= y.iloc[i * fold_size : (i + 1) * fold_size]
    x_train2 = pd.concat([x.iloc[:i * fold_size], x.iloc[(i + 1) * fold_size:]])
    y_train2 = pd.concat([y.iloc[:i * fold_size], y.iloc[(i + 1) * fold_size:]])

In [31]:
cv = KFold(n_splits=5, shuffle=False)  
rf = RandomForestClassifier(random_state=42)

params = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [None, 10, 20],      
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],    
    'class_weight': ['balanced']      
}

grid_search2 = GridSearchCV(
    estimator=rf,        
    param_grid=params,   
    scoring='balanced_accuracy',
    cv=cv,
    verbose=2,           
    n_jobs=4,           
)

grid_search2.fit(x_train2, y_train2)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(random_state=42), n_jobs=4,
             param_grid={'class_weight': ['balanced'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='balanced_accuracy', verbose=2)

In [32]:
best_params = grid_search1.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


In [34]:
rf = RandomForestClassifier(
    class_weight= 'balanced',
    max_depth= 10, 
    min_samples_leaf= 4, 
    min_samples_split= 10, 
    n_estimators= 100            
)

rf.fit(x_train2, y_train2)
y1_pred2 = rf.predict(x_test2)

balance = balanced_accuracy_score(y_test2, y1_pred2)
print(balance*100)

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


62.501563578798525


In [35]:
a, b, c, d, e = utils.print_metrics(y_test2, y1_pred2)

Accuracy: 0.85
Balanced Accuracy: 0.63
Precision: 0.90
Recall: 0.85
F1 Score: 0.87


In [36]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
dh

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20200819X80407,1,0.007710,0.005339,0.039732,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,1,0,0,1,0
1,20170704X12341,1,0.018236,0.005339,0.411220,0.400000,0.000000,0.25,0.783784,0.322581,...,0,1,0,0,0,0,0,1,1,0
2,20141107X32003,1,0.014458,0.005339,0.089816,0.000000,0.000000,0.25,0.783784,0.290323,...,0,1,0,0,0,0,0,1,1,0
3,20090202X02717,1,0.012294,0.005339,0.093661,0.000000,0.333333,0.25,0.810811,0.290323,...,0,0,0,1,0,1,0,0,1,0
4,20100806X95014,1,0.015434,0.000161,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,20220822105779,1,0.007705,0.005339,0.086957,0.040000,0.056667,0.25,0.783784,0.161290,...,0,1,0,0,0,1,0,0,1,0
3104,20130711X70752,1,0.012294,0.005339,0.027605,0.083333,0.000000,0.25,0.783784,0.387097,...,0,0,1,0,0,1,0,0,1,0
3105,20201026102189,1,0.060355,0.001768,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,1,0,0,0,0,1,0,0,1,0
3106,20160621X92619,1,0.011811,0.004447,0.517993,0.000000,0.000000,0.25,0.783784,0.096774,...,0,0,0,1,0,0,0,1,1,0


In [37]:
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["damage"]

In [38]:
y_h_pred = rf.predict(x_h)

In [39]:
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.85
Balanced Accuracy: 0.61
Precision: 0.89
Recall: 0.85
F1 Score: 0.87


In [ ]:
### Training logistic Regression on the training set with EV_HIGHEST_INJURY as the target variable
### steps followed:
# training the model
# using CV-HPT 
# trying cv-hpt best model on holdout set

In [40]:
target_column=['ev_highest_injury']
x_train, x_test, y_train, y_test=utils.dataset_traintest_split(df, target_column)

In [41]:
x_train = x_train.drop(columns=['ev_id','severity_class','damage'])
x_test = x_test.drop(columns=['ev_id','severity_class','damage'])

In [42]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, 
                            random_state=42,
                            class_weight='balanced')
rf.fit(x_train, y_train)

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [43]:
y_pred2 = rf.predict(x_test)

a, b, c, d, e = utils.print_metrics(y_test,y_pred2)

Accuracy: 0.77
Balanced Accuracy: 0.34
Precision: 0.65
Recall: 0.77
F1 Score: 0.68


In [44]:
x = df.drop(columns=['severity_class','ev_id','damage','ev_highest_injury'])
y = df[['ev_highest_injury']]
k = 5
fold_size = len(x) // k
for i in range(k):
    x_test2 = x.iloc[i * fold_size : (i + 1) * fold_size]
    y_test2= y.iloc[i * fold_size : (i + 1) * fold_size]
    x_train2 = pd.concat([x.iloc[:i * fold_size], x.iloc[(i + 1) * fold_size:]])
    y_train2 = pd.concat([y.iloc[:i * fold_size], y.iloc[(i + 1) * fold_size:]])

In [46]:
cv = KFold(n_splits=5, shuffle=False)  
rf = RandomForestClassifier(random_state=42)

params = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [None, 10, 20],      
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],    
    'class_weight': ['balanced']      
}

grid_search2 = GridSearchCV(
    estimator=rf,        
    param_grid=params,   
    scoring='balanced_accuracy',
    cv=cv,
    verbose=2,           
    n_jobs=4,           
)

grid_search2.fit(x_train2, y_train2)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(random_state=42), n_jobs=4,
             param_grid={'class_weight': ['balanced'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='balanced_accuracy', verbose=2)

In [47]:
best_params = grid_search1.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


In [48]:
rf = RandomForestClassifier(
    class_weight= 'balanced',
    max_depth= 10, 
    min_samples_leaf= 4, 
    min_samples_split= 10, 
    n_estimators= 100             
)

rf.fit(x_train2, y_train2)
y1_pred2 = rf.predict(x_test2)

balance = balanced_accuracy_score(y_test2, y1_pred2)
print(balance*100)

C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


46.08650333853319


In [49]:
a, b, c, d, e = utils.print_metrics(y_test2, y1_pred2)

Accuracy: 0.66
Balanced Accuracy: 0.46
Precision: 0.72
Recall: 0.66
F1 Score: 0.68


In [50]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
dh

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20200819X80407,1,0.007710,0.005339,0.039732,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,1,0,0,1,0
1,20170704X12341,1,0.018236,0.005339,0.411220,0.400000,0.000000,0.25,0.783784,0.322581,...,0,1,0,0,0,0,0,1,1,0
2,20141107X32003,1,0.014458,0.005339,0.089816,0.000000,0.000000,0.25,0.783784,0.290323,...,0,1,0,0,0,0,0,1,1,0
3,20090202X02717,1,0.012294,0.005339,0.093661,0.000000,0.333333,0.25,0.810811,0.290323,...,0,0,0,1,0,1,0,0,1,0
4,20100806X95014,1,0.015434,0.000161,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,20220822105779,1,0.007705,0.005339,0.086957,0.040000,0.056667,0.25,0.783784,0.161290,...,0,1,0,0,0,1,0,0,1,0
3104,20130711X70752,1,0.012294,0.005339,0.027605,0.083333,0.000000,0.25,0.783784,0.387097,...,0,0,1,0,0,1,0,0,1,0
3105,20201026102189,1,0.060355,0.001768,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,1,0,0,0,0,1,0,0,1,0
3106,20160621X92619,1,0.011811,0.004447,0.517993,0.000000,0.000000,0.25,0.783784,0.096774,...,0,0,0,1,0,0,0,1,1,0


In [51]:
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["damage"]

In [52]:
y_h_pred = rf.predict(x_h)

In [53]:
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.62
Balanced Accuracy: 0.29
Precision: 0.78
Recall: 0.62
F1 Score: 0.69
